<a href="https://colab.research.google.com/github/RioSatriatama/Algoritma-Informed-Search/blob/main/Algoritma_Informed_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# algoritma Informed Search (Pencarian Terinformasi)

# Nama : Fransiskus Xaverius Prasetyo Satriatama (Rio)

#Email : riosatriatama0801@gmail.com

#Github : https://github.com/RioSatriatama?tab=repositories

In [ ]:
import heapq

# --- Representasi Graf dan Heuristik ---
# Graf ini merepresentasikan kota-kota yang terhubung.
# Formatnya adalah: 'Kota': [('Tetangga1', Jarak_Asli), ('Tetangga2', Jarak_Asli), ...]
# Jarak_Asli ini adalah biaya g(n) atau cost dari satu node ke node tetangganya.
graf = {
    'Arad': [('Zerind', 75), ('Sibiu', 140), ('Timisoara', 118)],
    'Zerind': [('Arad', 75), ('Oradea', 71)],
    'Oradea': [('Zerind', 71), ('Sibiu', 151)],
    'Sibiu': [('Arad', 140), ('Oradea', 151), ('Fagaras', 99), ('Rimnicu Vilcea', 80)],
    'Timisoara': [('Arad', 118), ('Lugoj', 111)],
    'Lugoj': [('Timisoara', 111), ('Mehadia', 70)],
    'Mehadia': [('Lugoj', 70), ('Drobeta', 75)],
    'Drobeta': [('Mehadia', 75), ('Craiova', 120)],
    'Craiova': [('Drobeta', 120), ('Rimnicu Vilcea', 146), ('Pitesti', 138)],
    'Rimnicu Vilcea': [('Sibiu', 80), ('Craiova', 146), ('Pitesti', 97)],
    'Fagaras': [('Sibiu', 99), ('Bucharest', 211)],
    'Pitesti': [('Rimnicu Vilcea', 97), ('Craiova', 138), ('Bucharest', 101)],
    'Bucharest': [('Fagaras', 211), ('Pitesti', 101), ('Giurgiu', 90), ('Urziceni', 85)],
    'Giurgiu': [('Bucharest', 90)],
    'Urziceni': [('Bucharest', 85), ('Hirsova', 98), ('Vaslui', 142)],
    'Hirsova': [('Urziceni', 98), ('Eforie', 86)],
    'Eforie': [('Hirsova', 86)],
    'Vaslui': [('Urziceni', 142), ('Iasi', 92)],
    'Iasi': [('Vaslui', 92), ('Neamt', 87)],
    'Neamt': [('Iasi', 87)]
}

# Heuristik (h(n)): Estimasi jarak garis lurus dari setiap kota ke tujuan (Bucharest)
# Ini adalah "informasi" yang digunakan oleh algoritma.
heuristik = {
    'Arad': 366, 'Bucharest': 0, 'Craiova': 160, 'Drobeta': 242, 'Eforie': 161,
    'Fagaras': 176, 'Giurgiu': 77, 'Hirsova': 151, 'Iasi': 226, 'Lugoj': 244,
    'Mehadia': 241, 'Neamt': 234, 'Oradea': 380, 'Pitesti': 100, 'Rimnicu Vilcea': 193,
    'Sibiu': 253, 'Timisoara': 329, 'Urziceni': 80, 'Vaslui': 199, 'Zerind': 374
}

# --- Fungsi Bantuan untuk Rekonstruksi Jalur ---
def reconstruct_path(came_from, current):
    """Membangun kembali jalur dari node awal ke node tujuan."""
    total_path = [current]
    while current in came_from:
        current = came_from[current]
        total_path.insert(0, current)
    return total_path

# --- Implementasi Algoritma Greedy Best-First Search ---
def greedy_best_first_search(graph, start, goal, heuristic):
    """
    Algoritma Greedy Best-First Search.
    Hanya mempertimbangkan nilai heuristik (h(n)) untuk ekspansi.
    """
    # Priority queue untuk menyimpan node yang akan dikunjungi.
    # Format: (nilai_heuristik, nama_node)
    open_set = [(heuristic[start], start)]

    # Dictionary untuk melacak jalur
    came_from = {}

    # Set untuk melacak node yang sudah pernah ada di open_set
    # untuk menghindari duplikasi dengan prioritas lebih buruk
    open_set_nodes = {start}

    while open_set:
        # Ambil node dengan nilai heuristik terkecil
        _, current = heapq.heappop(open_set)

        if current == goal:
            return reconstruct_path(came_from, current)

        open_set_nodes.remove(current)

        for neighbor, _ in graph[current]:
            # Jika tetangga belum pernah dievaluasi, tambahkan ke open_set
            if neighbor not in open_set_nodes and neighbor not in came_from:
                came_from[neighbor] = current
                heapq.heappush(open_set, (heuristic[neighbor], neighbor))
                open_set_nodes.add(neighbor)

    return None # Jalur tidak ditemukan

# --- Implementasi Algoritma A* Search ---
def a_star_search(graph, start, goal, heuristic):
    """
    Algoritma A* Search.
    Mempertimbangkan f(n) = g(n) + h(n)
    g(n) = biaya dari awal ke node saat ini
    h(n) = biaya heuristik dari node saat ini ke tujuan
    """
    # Priority queue. Format: (f_cost, h_cost (untuk tie-breaking), nama_node)
    open_set = [(heuristic[start], heuristic[start], start)]

    # Dictionary untuk melacak jalur
    came_from = {}

    # g_cost menyimpan biaya dari awal ke setiap node
    g_cost = {node: float('inf') for node in graph}
    g_cost[start] = 0

    while open_set:
        # Ambil node dengan f_cost terkecil
        f_cost_current, _, current = heapq.heappop(open_set)

        if current == goal:
            total_cost = g_cost[goal]
            path = reconstruct_path(came_from, current)
            return path, total_cost

        # Iterasi melalui tetangga dari node saat ini
        for neighbor, distance in graph[current]:
            # Hitung g_cost tentatif ke tetangga
            tentative_g_cost = g_cost[current] + distance

            # Jika jalur baru ke tetangga ini lebih baik
            if tentative_g_cost < g_cost[neighbor]:
                came_from[neighbor] = current
                g_cost[neighbor] = tentative_g_cost
                f_cost = tentative_g_cost + heuristic[neighbor]
                heapq.heappush(open_set, (f_cost, heuristic[neighbor], neighbor))

    return None, None # Jalur tidak ditemukan

# --- Menjalankan Algoritma dan Menampilkan Hasil ---
start_node = 'Arad'
goal_node = 'Bucharest'

print(f"Mencari jalur dari '{start_node}' ke '{goal_node}'\n")

# Menjalankan Greedy Best-First Search
print("--- Greedy Best-First Search ---")
path_gbfs = greedy_best_first_search(graf, start_node, goal_node, heuristik)
if path_gbfs:
    print(f"Jalur ditemukan: {' -> '.join(path_gbfs)}")
else:
    print("Jalur tidak ditemukan.")

print("\n" + "="*40 + "\n")

# Menjalankan A* Search
print("--- A* Search ---")
path_astar, cost_astar = a_star_search(graf, start_node, goal_node, heuristik)
if path_astar:
    print(f"Jalur ditemukan: {' -> '.join(path_astar)}")
    print(f"Total biaya: {cost_astar}")
else:
    print("Jalur tidak ditemukan.")

Mencari jalur dari 'Arad' ke 'Bucharest'

--- Greedy Best-First Search ---


                 ┌────────────────────────────┐
                 │   Mulai Program            │
                 └───────────┬────────────────┘
                             │
                             ▼
              ┌────────────────────────────┐
              │ Inisialisasi Graf &        │
              │ Heuristik (h(n))           │
              └───────────┬────────────────┘
                          │
                          ▼
          ┌────────────────────────────┐
          │ Input node awal (start)    │
          │ dan node tujuan (goal)     │
          └───────────┬────────────────┘
                      │
                      ▼
        ┌───────────────────────────────┐
        │ Pilih Algoritma:              │
        │ 1. Greedy Best-First Search   │
        │ 2. A* Search                  │
        └──────────┬────────────────────┘
                   │
        ┌──────────┴──────────┐
        ▼                     ▼
 ┌──────────────────┐   ┌───────────────────┐
 │ Greedy Best-First │   │   A* Search       │
 │ Search            │   │                   │
 └───────┬───────────┘   └─────────┬─────────┘
         │                         │
         ▼                         ▼
┌───────────────────────┐   ┌─────────────────────────────┐
│ Buat open_set (PQ)    │   │ Buat open_set (PQ)           │
│ dengan node awal      │   │ dengan node awal             │
└─────────┬─────────────┘   └──────────┬──────────────────┘
          │                            │
          ▼                            ▼
┌────────────────────────────┐  ┌────────────────────────────┐
│ Ambil node dengan h(n)     │  │ Ambil node dengan f(n)=g+h │
│ terkecil dari open_set     │  │ terkecil dari open_set     │
└─────────┬──────────────────┘  └──────────┬─────────────────┘
          │                                 │
          ▼                                 ▼
┌────────────────────────────┐  ┌─────────────────────────────┐
│ Apakah node == goal?       │  │ Apakah node == goal?        │
└───────┬───────────┬────────┘  └───────┬───────────┬────────┘
        │           │                   │           │
       Tidak        Ya                 Tidak        Ya
        │           │                   │           │
        ▼           ▼                   ▼           ▼
┌────────────────┐  ┌───────────────────────────┐  ┌──────────────────────────┐
│ Ekspansi node: │  │ Rekonstruksi Jalur (path) │  │ Rekonstruksi Jalur (path)│
│ - Cek tetangga │  └───────────────┬───────────┘  │ + Hitung total biaya     │
│ - Tambahkan ke │                  │              └───────────┬──────────────┘
│   open_set     │                  │                          │
└───────┬────────┘                  │                          │
        │                            │                          │
        └─────────────── Loop ───────┘                          │
                                                                │
                                 ┌──────────────────────────────┘
                                 ▼
                     ┌───────────────────────────┐
                     │ Cetak hasil (jalur & cost) │
                     └───────────────┬───────────┘
                                     │
                                     ▼
                         ┌────────────────────────┐
                         │     Selesai Program    │
                         └────────────────────────┘


# Reference

- https://github.com/Caknoooo/Informed-Search-Algorithm

- https://www.geeksforgeeks.org/artificial-intelligence/informed-search-algorithms-in-artificial-intelligence/